In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1

In [2]:
prod = ['140A',
        '143A',
        '349A',
        '582A',
        '964A',]

In [3]:
test = ['029A',
        '980A',
        '179A',
        '456A',
        '379A',
       ]

In [11]:
from collections import deque
from functools import cache, lru_cache

# Numeric keypad layout
NUM_POS = {
    '7': (0,0), '8':(1,0), '9':(2,0),
    '4': (0,1), '5':(1,1), '6':(2,1),
    '1': (0,2), '2':(1,2), '3':(2,2),
    '0': (1,3), 'A':(2,3),
}
# Directional keypad layout
DIR_POS = {
    '^': (1,0), 'A':(2,0),
    '<': (0,1), 'v':(1,1), '>' :(2,1),
}


def check_grouped_button(path):
    if path == '':
        return True
    for i,ic in enumerate(path[:-1]):
        if ic != path[i+1] and path[i+1] in path[:i]:
            return False
    return True
    
def build_distances(pos_map):
    dists = {}
    for src in pos_map:
        d = {}
        dist_steps ={}
        sx, sy = pos_map[src]
        queue = deque([(sx, sy, "", 0)])
        seen = {(sx, sy): 0}
        while queue:
            x, y, path,steps = queue.popleft()
            
            if (x,y) in pos_map.values():
                tgt = next(k for k,v in pos_map.items() if v==(x,y))
                if check_grouped_button(path):
                    if tgt not in d.keys() or dist_steps[tgt] == steps:
                        dist_steps[tgt] = steps
                        try:
                            d[tgt].append(path)
                        except KeyError:
                            d[tgt] = [path]
                    
            for move, (dx, dy) in {'<':(-1,0),'>':(1,0),'^':(0,-1),'v':(0,1)}.items():
                nx, ny = x+dx, y+dy
                new_steps = steps + 1
                if (nx, ny) in pos_map.values() and ( (nx,ny) not in seen.keys() or new_steps <= seen[(nx,ny)] ):
                    seen[(nx,ny)] = new_steps
                    queue.append((nx, ny, path + move, new_steps))
        dists[src] = d
    return dists

# Compute movement maps for both keypads
NUM_PATHS = build_distances(NUM_POS)
DIR_PATHS = build_distances(DIR_POS)


#### Clean up for optimal orientation
def select_optimal_path(paths): 
    single_paths ={}
    for s, end in paths.items():
        new_end={}
        for e, path in end.items():
            if len(path) == 2:
                if path[0].endswith('^'):
                    new_end[e] = path[0]
                elif path[1].endswith('>'):
                    new_end[e] = path[1]
                elif path[0].startswith('<'):
                    new_end[e] = path[0]
                elif path[1].startswith('<'):
                    new_end[e] = path[1]
                else:
                    new_end[e] = path[1]
            else:
                new_end[e] = path[0]
        single_paths[s] = new_end
    return single_paths
DIR_PATHS = select_optimal_path(DIR_PATHS)
NUM_PATHS = select_optimal_path(NUM_PATHS)

# Compute Calculate Buttons necessary before

@cache
def generate_upper_sequence(code, numbers=False):
    shortest_paths = NUM_PATHS if numbers else DIR_PATHS
    upper_return=[]
        
    full_path = 'A' + code
    upper_candidate = ''
    for s, e in zip(full_path[:-1], full_path[1:]):
        upper_candidate = upper_candidate+shortest_paths[s][e]+'A'
    return upper_candidate

In [12]:
def part_one_complexity(codes):
    complexity=0
    for code in codes:
        r1 = generate_upper_sequence(code, True)
        r2 =  generate_upper_sequence(r1)
        person =  generate_upper_sequence(r2)

        shortest_dist = len(person)

        print(shortest_dist,int(code.strip('A')))
        
        complexity += shortest_dist*int(code.strip('A'))
    return complexity

In [13]:
%%time
part_one_complexity(test)

68 29
60 980
68 179
64 456
64 379
CPU times: user 820 μs, sys: 391 μs, total: 1.21 ms
Wall time: 897 μs


126384

In [7]:
%%time
part_one_complexity(prod)

70 140
72 143
72 349
68 582
72 964
CPU times: user 294 μs, sys: 8 μs, total: 302 μs
Wall time: 301 μs


154208

# Part 2

In [17]:
def find(string, char):
    return [i for i, ltr in enumerate(string) if ltr == char]
    
def chunk_string(code):
    longer = 'A'+code
    alocs = find(longer,'A')
    upper=''
    for i,j in zip( alocs, alocs[1:]):
        upper = upper + generate_upper_sequence(longer[i+1: j+1])
    return upper 


#seen_code = {}
@cache
def recurse_segment(code: str, depth: int) -> str:
    """
    Recursively builds all possible minimal-length input sequences
    needed at layer `depth` to produce `code` at the bottom.
    """
    
    longer = 'A'+code
    alocs = find(longer,'A')

    total_len=0
    if depth == 0 : 
        for i,j in zip( alocs, alocs[1:]):
            lower_code = longer[i+1: j+1]
            next_seg =  generate_upper_sequence(lower_code) 
            total_len += len(next_seg)
        return total_len
        
    for i,j in zip( alocs, alocs[1:]):
        lower_code = longer[i+1: j+1]
        key = (lower_code, depth)
        
        next_seg =  generate_upper_sequence(lower_code) 
        
        total_len += recurse_segment(next_seg, depth-1)
    return total_len
    


In [20]:
def part_two_complexity(codes, nrobots=25):
    complexity = 0
    for code in codes:
        # Clean and format the code
        clean_code = code.strip()
        if not clean_code.endswith('A'):
            clean_code += 'A'

        r1 = generate_upper_sequence(code, True)
        path_length =recurse_segment(r1, nrobots-1)

        print(path_length,int(code.strip('A')))
        
        complexity += path_length*int(code.strip('A'))
    return complexity


In [21]:
%%time
part_two_complexity(test,2)

68 29
60 980
68 179
64 456
64 379
CPU times: user 564 μs, sys: 18 μs, total: 582 μs
Wall time: 577 μs


126384

In [26]:
%%time
part_two_complexity(prod,25)

99333974668 140
101779364540 143
101257214304 349
98217357614 582
97169520992 964
CPU times: user 202 μs, sys: 8 μs, total: 210 μs
Wall time: 209 μs


214633893742472

In [40]:
def simulate_robot_layer_output(code_sequence, keypad_layout, start='A'):
    # Build reverse map from position to key
    pos_to_key = {v: k for k, v in keypad_layout.items()}
    x, y = keypad_layout[start]
    output = ''
    for ch in code_sequence:
        if ch in {'<', '>', '^', 'v'}:
            dx, dy = {'<': (-1, 0), '>': (1, 0), '^': (0, -1), 'v': (0, 1)}[ch]
            nx, ny = x + dx, y + dy
            if (nx, ny) in pos_to_key:
                x, y = nx, ny
            else:
                raise ValueError(f"Invalid move {ch} from position {(x, y)}")
        elif ch == 'A':
            output += pos_to_key[(x, y)]
        else:
            raise ValueError(f"Invalid input character: {ch}")
    return output


In [42]:
simulate_robot_layer_output('<v<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A', DIR_POS)

'<A>Av<<AA>^AA>AvAA^A<vAAA>^A'

In [45]:
simulate_robot_layer_output('v<<A>>^AvA^Av<<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^AA<A>A<vA<A>>^AAA<A>vA^A', DIR_POS)

'<A>A<AAv<AA>>^AvAA^Av<AAA^>A'

In [59]:
start='<v<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A'
print(len(start), start)
tmp = simulate_robot_layer_output(start, DIR_POS)
print(len(tmp), tmp)
tmp = simulate_robot_layer_output(tmp, DIR_POS)
print(len(tmp), tmp)
simulate_robot_layer_output(tmp, NUM_POS)

64 <v<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A
28 <A>Av<<AA>^AA>AvAA^A<vAAA>^A
14 ^A<<^^A>>AvvvA


'379A'

In [60]:
start='v<<A>>^AvA^Av<<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^AA<A>A<vA<A>>^AAA<A>vA^A'
print(len(start), start)
tmp = simulate_robot_layer_output('v<<A>>^AvA^Av<<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^AA<A>A<vA<A>>^AAA<A>vA^A', DIR_POS)
print(len(tmp), tmp)
tmp = simulate_robot_layer_output(tmp, DIR_POS)
print(len(tmp), tmp)
simulate_robot_layer_output(tmp, NUM_POS)

68 v<<A>>^AvA^Av<<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^AA<A>A<vA<A>>^AAA<A>vA^A
28 <A>A<AAv<AA>>^AvAA^Av<AAA^>A
14 ^A^^<<A>>AvvvA


'379A'